In [147]:
## Import packages
import os
import csv
import random
import math
import ntpath
import numpy as np
import pandas as pd
import time
from pathlib import Path
from scipy.io import wavfile

import tensorflow as tf
from tensorflow import gfile
from tensorflow import logging

In [8]:
import sys
sys.path.append('./vggish')

import vggish_input
import vggish_postprocess
import vggish_params
import vggish_slim

In [19]:
serval_data_folder = "../data/"

In [20]:
flags = tf.app.flags
FLAGS = flags.FLAGS

delattr(flags.FLAGS, 'f')
flags.DEFINE_string('f', '', 'kernel')

delattr(flags.FLAGS, 'audio_embedding_feature_name')
flags.DEFINE_string(
    'audio_embedding_feature_name', 'audio_embedding',
    'Path to the VGGish checkpoint file.')

## De volgende bestanden komen uit het eerdere mapje 'models'. Nu heb ik ze in yt8m gezet.
pca_params_vggish_model_folder = serval_data_folder + "vggish_model/models/"
print('Files in vggish model directory: (Expecting vggish_..., youtube_...)')
print(os.listdir(pca_params_vggish_model_folder))

delattr(flags.FLAGS, 'pca_params')
flags.DEFINE_string(
    'pca_params', pca_params_vggish_model_folder + 'vggish_pca_params.npz',
    'Path to the VGGish PCA parameters file.')

delattr(flags.FLAGS, 'checkpoint')
flags.DEFINE_string(
    'checkpoint', pca_params_vggish_model_folder + 'vggish_model.ckpt',
    'Path to the VGGish checkpoint file.')

# flags.DEFINE_string(
#    'yt_checkpoint', 'models/youtube_model.ckpt',
#    'Path to the VGGish checkpoint file.')

Files in vggish model directory: (Expecting vggish_..., youtube_...)
['class_labels_indices.csv', 'vggish_model.ckpt', 'vggish_pca_params.npz', 'youtube_model.ckpt.data-00000-of-00001', 'youtube_model.ckpt.index', 'youtube_model.ckpt.meta']


In [134]:
wav_samples_custom_path = serval_data_folder + "wav_samples_custom/"
wav_samples_custom_labels_csv = wav_samples_custom_path + "wav_samples_custom_labels.csv"
target_wav_samples_custom_all_enumerated_csv = wav_samples_custom_path + "wav_samples_custom_all_enumerated_and_labeled.csv"

target_folder_tfrecords = serval_data_folder + "tf_records_samples/"



# label = "gunshots"
# label = 'bush_wenger'
# label = "elephants_wenger"
# label = 'elephants_angela'
#serval_class_labels_csv = './labels/custom/jungle/class_labels_indices_jungle_v2.csv'
#custom_labels_csv = './labels/custom/jungle/meta-data-elephant-adventures.csv'
#source_wolder_wavs = "../../data/audio/serval-data/wav-16k-16khz/elephants-angela"
#target_folder_tfrecords = "../../data/audio/serval-data/tfrecords/elephants-angela"

In [56]:
wav_samples_class_labels = pd.read_csv(wav_samples_custom_labels_csv, sep=";")
print(wav_samples_class_labels)

   label                      folder_name                     description  \
0   1000             3_distance_9mm_shots        3 Distance 9mm gun shots   
1   1001                     556_gunshots                    556 gunshots   
2   1002                 a_brommer_alarms                   Brommer Alarm   
3   1003                       b_brommers                         Brommer   
4   1004                        c_claxons                         Claxons   
5   1005      k_dichtklappende_autodeuren       Dichtklappende autodeuren   
6   1006  m_luide_mensen_schreeuwen_en_zo  Luide mensen (schreeuwen, enz)   
7   1007                        o_motoren                         Motoren   
8   1008                 t_terrasgeluiden   Terrasgeluide (stille mensen)   
9   1009              v_versterkte_muziek               Versterkte muziek   

   enable  
0       1  
1       1  
2       1  
3       1  
4       1  
5       1  
6       1  
7       1  
8       1  
9       1  


In [123]:
# Enumerate all the available wav files, label them and store a data-frame of the result
def collectAndLabelSamples(full_path_name, label, description, class_name):
    ## Read and rewrite all test files
    files = gfile.Glob(str(full_path_name + "*.wav"))
    
    df = pd.DataFrame(columns=['index', 'label', 'description', 'file_path', 'filename', 'class_name'])
    count = 1
    for file in files:
        df_row = pd.DataFrame({'index':count, 'label':[label], 'description':[description], 'file_path':[file], 'filename':[ntpath.basename(file)], 'class_name':[class_name]})
        df = df.append(df_row, ignore_index = True)
        count = count + 1
    return(df)

In [124]:
## Enumerate all the available wav files, label them and store a data-frame of the result
df_all_wav_samples = pd.DataFrame(columns=['index', 'label', 'description', 'file_path', 'filename', 'class_name'])
for index, row in wav_samples_class_labels.iterrows():
    if row['enable'] == 1:
        print('Processing record ', index, '; label=', row.label, ', description=', row.description, ', path=', row.folder_name, sep='')
        df_row = collectAndLabelSamples(wav_samples_custom_path + row.folder_name + '/', row.label, row.description, row.folder_name)
        df_all_wav_samples = df_all_wav_samples.append(df_row, ignore_index = True)
    else:
        print('Skipping record ', index, '; label=', row.label, ', description=', row.description, ', path=', row.folder_name, sep='')

Processing record 0; label=1000, description=3 Distance 9mm gun shots, path=3_distance_9mm_shots
Processing record 1; label=1001, description=556 gunshots, path=556_gunshots
Processing record 2; label=1002, description=Brommer Alarm, path=a_brommer_alarms
Processing record 3; label=1003, description=Brommer, path=b_brommers
Processing record 4; label=1004, description=Claxons, path=c_claxons
Processing record 5; label=1005, description=Dichtklappende autodeuren, path=k_dichtklappende_autodeuren
Processing record 6; label=1006, description=Luide mensen (schreeuwen, enz), path=m_luide_mensen_schreeuwen_en_zo
Processing record 7; label=1007, description=Motoren, path=o_motoren
Processing record 8; label=1008, description=Terrasgeluide (stille mensen), path=t_terrasgeluiden
Processing record 9; label=1009, description=Versterkte muziek, path=v_versterkte_muziek


In [125]:
## Write df_all_wav_samples to csv
print(df_all_wav_samples.head())
df_all_wav_samples.to_csv(target_wav_samples_custom_all_enumerated_csv, sep=';')


  index label               description  \
0     1  1000  3 Distance 9mm gun shots   
1     2  1000  3 Distance 9mm gun shots   
2     3  1000  3 Distance 9mm gun shots   
3     4  1000  3 Distance 9mm gun shots   
4     5  1000  3 Distance 9mm gun shots   

                                           file_path  \
0  ..\data\wav_samples_custom\3_distance_9mm_shot...   
1  ..\data\wav_samples_custom\3_distance_9mm_shot...   
2  ..\data\wav_samples_custom\3_distance_9mm_shot...   
3  ..\data\wav_samples_custom\3_distance_9mm_shot...   
4  ..\data\wav_samples_custom\3_distance_9mm_shot...   

                   filename            class_name  
0   shot distance9mm.06.wav  3_distance_9mm_shots  
1   shot distance9mm.10.wav  3_distance_9mm_shots  
2  shot distance9mm.105.wav  3_distance_9mm_shots  
3  shot distance9mm.106.wav  3_distance_9mm_shots  
4  shot distance9mm.107.wav  3_distance_9mm_shots  


In [126]:
## Read all samples from disk in case you would like to change some by hand.
df_all_wav_samples = []
df_all_wav_samples = pd.read_csv(target_wav_samples_custom_all_enumerated_csv, sep=";")
print(df_all_wav_samples.head())

   Unnamed: 0  index  label               description  \
0           0      1   1000  3 Distance 9mm gun shots   
1           1      2   1000  3 Distance 9mm gun shots   
2           2      3   1000  3 Distance 9mm gun shots   
3           3      4   1000  3 Distance 9mm gun shots   
4           4      5   1000  3 Distance 9mm gun shots   

                                           file_path  \
0  ..\data\wav_samples_custom\3_distance_9mm_shot...   
1  ..\data\wav_samples_custom\3_distance_9mm_shot...   
2  ..\data\wav_samples_custom\3_distance_9mm_shot...   
3  ..\data\wav_samples_custom\3_distance_9mm_shot...   
4  ..\data\wav_samples_custom\3_distance_9mm_shot...   

                   filename            class_name  
0   shot distance9mm.06.wav  3_distance_9mm_shots  
1   shot distance9mm.10.wav  3_distance_9mm_shots  
2  shot distance9mm.105.wav  3_distance_9mm_shots  
3  shot distance9mm.106.wav  3_distance_9mm_shots  
4  shot distance9mm.107.wav  3_distance_9mm_shots  


In [102]:
## Proportie train/test
pct_train = 0.6

In [103]:
## Voor alle wav-files in een map, gebeurt het volgende:

## Eerst wordt het wav-file ingelezen en de bijbehorende label(s) opgezocht in een csv-bestand
## Als video-id wordt een willekeurig id'tje gegeven (in dit geval allemaal dezelfde)

## Dan parsen we wav-file naar embeddings: 
# (dit gebeurt door vggish_input.wavfile_to_example aan te roepen)
# Stap 1a: lezen van wav-file, input is array met samples die db aanduiden. Ook sample rate (per sec) wordt gelezen
# Stap 1b: Bij 2d array (stereo, ipv mono) bereken gemiddelde, daarna normaliseren (delen door 32.768)
# Stap 2: Bepaal examples in vorm [batch size, num frames, num bands].
    # Hierbij worden voor verschillende batches (omdat alles tegelijk niet in 1x in NN kan),
    # een log mel spectrogram gemaakt (in vorm [num_frames, num_bands])
# Stap 3: Bepaal features: nu wordt de embedding laag gemaakt (PCA-components, discreet maken etc)
    # Hiervoor worden model-parameters opgehaald die eerder zijn opgeslagen
    
## Daarna wordt een sequence example gemaakt (in getSequenceExample) en het als tf-records weggeschreven

In [104]:
## Function that takes examples from wav-file as input and returns a sequence example

def getSequenceExample(examples_batch, labels, video_id=[b'-1LrH01Ei1w']):
    with tf.Graph().as_default(), tf.Session() as sess:

        # Prepare a postprocessor to munge the model embeddings.
        pproc = vggish_postprocess.Postprocessor(FLAGS.pca_params)
    
        # Define the model: load the checkpoint and locate input and output tensors
        # Input: [batch_size, num_frames, num_bands] 
        # where [num_frames, num_bands] represents log-mel-scale spectrogram
        # Output: embeddings
        vggish_slim.define_vggish_slim(training=False)
        vggish_slim.load_vggish_slim_checkpoint(sess, FLAGS.checkpoint)

        features_tensor = sess.graph.get_tensor_by_name(
            vggish_params.VGGISH_INPUT_TENSOR_NAME)
        embedding_tensor = sess.graph.get_tensor_by_name(
            vggish_params.VGGISH_OUTPUT_TENSOR_NAME)

        # Run inference and postprocessing.
        [embedding_batch] = sess.run([embedding_tensor],
                                     feed_dict={features_tensor: examples_batch})
        print(embedding_batch.shape)
        if(embedding_batch.shape!=10):
            print("*****************************************************************")

        postprocessed_batch = pproc.postprocess(embedding_batch)
        #print(postprocessed_batch)


        ## Maak labels en video-id voor in de example
        label_feat = tf.train.Feature(int64_list=tf.train.Int64List(value=labels))
        videoid_feat = tf.train.Feature(bytes_list=tf.train.BytesList(value=video_id))

        ## Maak sequence example
        seq_example = tf.train.SequenceExample(
            context = tf.train.Features(feature={"labels": label_feat, "video_id": videoid_feat}),
            feature_lists = tf.train.FeatureLists(
                feature_list={
                    FLAGS.audio_embedding_feature_name:
                        tf.train.FeatureList(
                            feature=[
                                tf.train.Feature(
                                    bytes_list=tf.train.BytesList(
                                        value=[embedding.tobytes()]))
                                for embedding in postprocessed_batch
                            ]
                        )
                }
            )
        )
        
    return(seq_example)

In [167]:
def encode_wav_files(wav_files, df_labels, output_directory, class_name):
    tf_record_writer = tf.python_io.TFRecordWriter(str(output_directory + class_name + '.tfrecord'))
    
    start = time.time()
    count = 0
    for file in wav_files:
        print("[INFO]: Count=", count, "/", len(wav_files), "; Time=", round((time.time() - start)), sep='')
        print("Filename: " + file, sep='')

        ## Fetch labels
        df_sample_labels = df_labels.loc[df_labels.filename == ntpath.basename(file)]
        if len(df_sample_labels) == 0:
            print('[ERROR]: Found sample without labels; filename=', file, sep='')
        else:
            ## .. encode - Part 1
            examples_batch = vggish_input.wavfile_to_examples(file)
            
            if(examples_batch.shape[0]>10):
                nr_sec = examples_batch.shape[0]
                print(nr_sec)
                start = int(math.floor((nr_sec-10)/2))
                print("start: " + str(start))
                end = int(nr_sec-math.ceil((nr_sec-10)/2))
                print("end: " + str(end))
                examples_batch = examples_batch[start:end, :, :]
                print(examples_batch.shape)
            
            ## .. encode - Part 2
            seq_example = getSequenceExample(examples_batch, df_sample_labels.label.unique())
            
            ## .. write to file
            tf_record_writer.write(seq_example.SerializeToString())
            
            ## .. done
            count = count + 1

    tf_record_writer.close()

In [168]:
start = time.time()
for class_name in df_all_wav_samples.class_name.unique():
    df_class_wav_samples = df_all_wav_samples.loc[df_all_wav_samples.class_name == class_name]
    
    print('Processing: ', class_name, '; sample_count=', len(df_class_wav_samples), sep='')
    print("[INFO]: Start time: ", round((time.time() - start)))
    encode_wav_files(df_class_wav_samples.file_path, df_all_wav_samples, target_folder_tfrecords, class_name)
    print("[INFO]: End time: ", round((time.time() - start)))
    

Processing: 3_distance_9mm_shots; sample_count=178
[INFO]: Start time:  0
[INFO]: Count=0/178; Time=0
Filename: ..\data\wav_samples_custom\3_distance_9mm_shots\shot distance9mm.06.wav
INFO:tensorflow:Restoring parameters from ../data/vggish_model/models/vggish_model.ckpt
(5, 128)
*****************************************************************
[INFO]: Count=1/178; Time=7
Filename: ..\data\wav_samples_custom\3_distance_9mm_shots\shot distance9mm.10.wav
INFO:tensorflow:Restoring parameters from ../data/vggish_model/models/vggish_model.ckpt
(5, 128)
*****************************************************************
[INFO]: Count=2/178; Time=12
Filename: ..\data\wav_samples_custom\3_distance_9mm_shots\shot distance9mm.105.wav
INFO:tensorflow:Restoring parameters from ../data/vggish_model/models/vggish_model.ckpt
(5, 128)
*****************************************************************
[INFO]: Count=3/178; Time=18
Filename: ..\data\wav_samples_custom\3_distance_9mm_shots\shot distance9mm.1

In [16]:
## Lees stuk voor stuk alle wav-files in
## Zoek het bijbehorende label op in een csv-bestand
## Bepaal de embeddings

# Prepare a record writer to store the postprocessed embeddings.
## Trainset
if(label == "gunshots"):
    train_tfrecord = str(tfrecord_path + 'gun_train.tfrecord')
    test_tfrecord = str(tfrecord_path + 'gun_test.tfrecord')
elif(label == "elephants_angela"):
    train_tfrecord = str(tfrecord_path + 'elephant_angela_train.tfrecord')
    test_tfrecord = str(tfrecord_path + 'elephant_angela_test.tfrecord')
elif(label == "background_angela"):
    train_tfrecord = str(tfrecord_path + 'background_angela_train.tfrecord')
    test_tfrecord = str(tfrecord_path + 'background_angela_test.tfrecord')
elif(label == "elephants_wenger"):
    train_tfrecord = str(tfrecord_path + 'elephant_wenger_train.tfrecord')
    test_tfrecord = str(tfrecord_path + 'elephant_wenger_test.tfrecord')
elif(label == "bush_wenger"):
    train_tfrecord = str(tfrecord_path + 'bush_wenger_train.tfrecord')
    test_tfrecord = str(tfrecord_path + 'bush_wenger_test.tfrecord')

    
    
train_writer = tf.python_io.TFRecordWriter(train_tfrecord)
test_writer = tf.python_io.TFRecordWriter(test_tfrecord)

## Read and rewrite all test files
files = gfile.Glob(str(wavfile_path + "*.wav"))

count = 1
for file in files:
    
    print("file nr: " + str(count))
    print("Filename: " + file)
    
    count += 1
    
    #print("Filename: " + str(file))
   
    ## Find labels belonging to wav-file
    if(label=="gunshots"):
        labels = getLabels(file)
        if(labels[0]==11):
            nr_9mm += 1
        elif(labels[0]==12):
            nr_556 += 1
    elif(label == "elephants_wenger"):
        labels = [13]
        elephants += 1
    elif(label == "bush_wenger"):
        labels = [14]
        bush += 1
    elif(label == "elephants_angela"):
        labels = [15]
        elephants_angela += 1
    elif(label == "background_angela"):
        labels = [16]
        elephants_angela += 1
    
    ## This function reads the wav file and converts the samples into np arrays of [batch size, num frames, num bands]
    #examples_batch = vggish_input.wavfile_to_examples(str(FLAGS.wavfile_path + wav_file))
    examples_batch = vggish_input.wavfile_to_examples(file)
    
    if(examples_batch.shape[0]>10):
        nr_sec = examples_batch.shape[0]
        print(nr_sec)
        start = int(math.floor((nr_sec-10)/2))
        print("start: " + str(start))
        end = int(nr_sec-math.ceil((nr_sec-10)/2))
        print("end: " + str(end))
        examples_batch = examples_batch[start:end, :, :]
        print(examples_batch.shape)
    
    #print("Examples shape: " + str(examples_batch.shape))
   
    seq_example = getSequenceExample(examples_batch, labels)
    
    rand = random.random()
    if rand <= pct_train:
        train_writer.write(seq_example.SerializeToString())
    else:
        test_writer.write(seq_example.SerializeToString())
    
train_writer.close()
test_writer.close()

file nr: 1
Filename: elephants\angela_background\BackgroundEFAF2011A008-102.WAV
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
(10, 128)
*****************************************************************
file nr: 2
Filename: elephants\angela_background\BackgroundEFAF2011A008-165.WAV
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
(10, 128)
*****************************************************************
file nr: 3
Filename: elephants\angela_background\BackgroundEFAF2011A008-232.WAV
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
(10, 128)
*****************************************************************
file nr: 4
Filename: elephants\angela_background\BackgroundEFAF2011A008-326.WAV
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
(10, 128)
*****************************************************************
file nr: 5
Filename: elephants\angela_background\BackgroundEFAF2011A009-146.WAV
INFO:tensorflow:Restoring pa

In [10]:
print("nr 9mm: " + str(nr_9mm))
print("nr 556: " + str(nr_556))

nr 9mm: 0
nr 556: 0


In [16]:
print("nr elephants: " + str(elephants))

nr elephants: 3717


In [20]:
print("nr bush: " + str(bush))

nr bush: 1540


In [10]:
print("nr gun google: " + str(gun_google))

nr gun google: 414
